# 📜 Projeto Final - Capacitação IA (Ciclo 3)
# 🎓 Alunos: Filipe da Silva Rodrigues e Rodrigo Serafim Floriano da Silva

## 📚 Bibliotecas Necessárias

In [1]:
# Instalação de bibliotecas necessárias para execução do código
# !pip install numpy pandas scikit-learn mlflow xgboost lightgbm catboost tpot pytorch auto-sklearn --quiet

In [6]:
# Tratamento de Dataset e Métricas

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.kernel_approximation import RBFSampler

# Modelos de Treinamento

# Classificadores básicos
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Classificadores avançados
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# AutoML
from tpot import TPOTClassifier

# Armazenamento e Análise de Modelos

import mlflow
import mlflow.sklearn
from mlflow.pyfunc import PythonModel
from sklearn.pipeline import Pipeline

# Terminal

import os
import warnings
warnings.filterwarnings("ignore")


---

👾 **Dataset de Classificação - Kaggle: Water Quality**

Esse dataframe é um conjunto de dados que contém informações sobre a qualidade da água e sua potabilidade. As variáveis são:

- `ph`: o valor do pH da água (0 a 14).
- `Hardness`: a capacidade da água de precipitar sabão em mg/L.
- `Solids`: sólidos totais dissolvidos em ppm.
- `Chloramines`: quantidade de cloraminas em ppm.
- `Sulfate`: quantidade de sulfatos dissolvidos em mg/L.
- `Conductivity`: condutividade elétrica da água em μS/cm.
- `Organic_carbon`: quantidade de carbono orgânico em ppm.
- `Trihalomethanes`: quantidade de trihalometanos em μg/L.
- `Turbidity`: medida da propriedade de emissão de luz da água em NTU.
- `Potability`: indica se a água é segura para consumo humano (1 = Potável, 0 = Não potável).

✅ **Objetivo:** Prever se a água é potável ou não com base nas características coletadas.

---


In [3]:
# Carregar o dataset
url = 'water_potability.csv'
dataset = pd.read_csv(url)

# Analisar o dataset
print('\nInformações do Dataset:\n')
print(dataset.info())

print('\nVerificar Valores Nulos:\n')
print(dataset.isnull().sum())

# Exibir o dataset original
print('\nDataset Original:\n')
print(dataset.head())

# Criar uma cópia do dataset para efetuar os devidos tratamentos
df = dataset.copy()

# Lidar com dados quem possuem valores nulos
imputer = SimpleImputer(strategy='mean')
columns_to_normalize = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']
df[columns_to_normalize] = imputer.fit_transform(df[columns_to_normalize])

# Normalizar os dados (0 a 1)
scaler = MinMaxScaler()  # Alternativa: StandardScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Separar os dados para análise de features e target
target = df['Potability'].copy()
features = df.drop('Potability', axis=1).copy()

print("\nDistribuição de Classes do Dataset:\n")
print(target.value_counts())

# Separar dados para treino e teste
X_train, x_test, Y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=None, stratify=target)

# Balanceamento de classes com SMOTE no conjunto de treinamento
smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(X_train, Y_train)


print(f"\nTamanho do conjunto de treinamento: {x_train.shape}")
print(f"Tamanho do conjunto de teste: {x_test.shape}")



Informações do Dataset:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB
None

Verificar Valores Nulos:

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
d

## 🧪 Experimentos no MLFLOW

In [4]:


models = {
    "TPOT AutoML": [
        # Configuração com foco em exploração profunda e diversidade elevada
        {
            "generations": 25,            # Aumenta o número de gerações para uma exploração mais profunda do espaço de hiperparâmetros
            "population_size": 200,       # Maior diversidade de indivíduos na população, aumentando a capacidade de encontrar boas soluções
            "verbosity": 2,               # Exibe informações detalhadas durante o processo de evolução
            "scoring": "f1_weighted",     # F1 Score como métrica principal para otimização do modelo
            "random_state": 42,           # Garante reprodutibilidade dos resultados
            "n_jobs": -1                  # Utiliza todos os núcleos de processamento disponíveis para acelerar a execução
        },

        # Configuração com foco em exploração profunda e diversidade máxima
        {
            "generations": 30,            # Aumenta o número de gerações para uma exploração mais profunda do espaço de hiperparâmetros
            "population_size": 600,       # Diversidade máxima para explorar uma grande variedade de soluções
            "verbosity": 2,               # Detalha o progresso de cada geração
            "scoring": "f1_weighted",     # F1 Score como métrica principal para otimização do modelo
            "random_state": 42,           # Garantia de resultados consistentes entre execuções
            "n_jobs": -1                  # Maximiza o uso de recursos computacionais para acelerar o processo de busca
        },
    ],
    "Stacking Classifier": [
         # Combinação leve e eficiente
        {
            "estimators": [
                ("rf_pipeline", Pipeline([
                    ("scaler", MaxAbsScaler()),
                    ("classifier", RandomForestClassifier(
                        bootstrap=True,
                        criterion="gini",
                        max_features=1.0,
                        min_samples_leaf=8,
                        min_samples_split=6,
                        n_estimators=100,
                        random_state=42
                    ))
                ])),
                ("et1_pipeline", Pipeline([
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="entropy",
                        max_features=0.55,
                        min_samples_leaf=1,
                        min_samples_split=4,
                        n_estimators=100,
                        random_state=42
                    ))
                ])),
                ("et2_pipeline", Pipeline([
                    ("scaler", MaxAbsScaler()),
                    ("poly_features", PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="entropy",
                        max_features=0.4,
                        min_samples_leaf=1,
                        min_samples_split=2,
                        n_estimators=100,
                        random_state=42
                    ))
                ])),
                # Adicionando novos pipelines com RBFSampler e PolynomialFeatures
                ("et3_pipeline", Pipeline([
                    ("scaler", MaxAbsScaler()),
                    ("rbf_sampler", RBFSampler(gamma=0.5)),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="gini",
                        max_features=0.25,
                        min_samples_leaf=1,
                        min_samples_split=6,
                        n_estimators=100,
                        random_state=42
                    ))
                ])),
                ("et4_pipeline", Pipeline([
                    ("scaler", MaxAbsScaler()),
                    ("poly_features", PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="gini",
                        max_features=0.25,
                        min_samples_leaf=1,
                        min_samples_split=3,
                        n_estimators=100,
                        random_state=42
                    ))
                ]))
            ],
            "final_estimator": RandomForestClassifier(n_estimators=50, random_state=42)
        },

           # Configuração robusta com pipelines diversificados
        {
            "estimators": [
                ("rf_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("classifier", RandomForestClassifier(
                        bootstrap=True,
                        criterion="gini",
                        max_features=0.8,
                        min_samples_leaf=4,
                        min_samples_split=5,
                        n_estimators=200,
                        random_state=42
                    ))
                ])),
                ("et_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="entropy",
                        max_features=0.65,
                        min_samples_leaf=2,
                        min_samples_split=3,
                        n_estimators=200,
                        random_state=42
                    ))
                ])),
                ("gb_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("poly_features", PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)),
                    ("classifier", GradientBoostingClassifier(
                        n_estimators=150,
                        learning_rate=0.05,
                        max_depth=5,
                        min_samples_split=4,
                        random_state=42
                    ))
                ])),
                ("hgb_pipeline", Pipeline([
                    ("scaler", MinMaxScaler()),
                    ("classifier", HistGradientBoostingClassifier(
                        max_iter=300,
                        learning_rate=0.08,
                        max_depth=6,
                        random_state=42
                    ))
                ])),
                ("cb_pipeline", Pipeline([
                    ("scaler", MinMaxScaler()),
                    ("classifier", CatBoostClassifier(
                        iterations=200,
                        learning_rate=0.1,
                        depth=6,
                        verbose=False,
                        allow_writing_files=False,
                        random_state=42
                    ))
                ])),
                # Novos pipelines robustos com RBFSampler e PolynomialFeatures
                ("et3_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("rbf_sampler", RBFSampler(gamma=0.3)),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="gini",
                        max_features=0.35,
                        min_samples_leaf=2,
                        min_samples_split=4,
                        n_estimators=150,
                        random_state=42
                    ))
                ])),
                ("et4_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("poly_features", PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)),
                    ("classifier", ExtraTreesClassifier(
                        bootstrap=False,
                        criterion="gini",
                        max_features=0.3,
                        min_samples_leaf=1,
                        min_samples_split=2,
                        n_estimators=150,
                        random_state=42
                    ))
                ])),
                ("xgb_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("classifier", XGBClassifier(
                        n_estimators=300,
                        learning_rate=0.05,
                        max_depth=6,
                        min_child_weight=3,
                        subsample=0.9,
                        random_state=42
                    ))
                ])),
                ("lgbm_pipeline", Pipeline([
                    ("scaler", StandardScaler()),
                    ("classifier", LGBMClassifier(
                        n_estimators=200,
                        learning_rate=0.1,
                        max_depth=6,
                        num_leaves=40,
                        random_state=42
                    ))
                ]))
            ],
            "final_estimator": LogisticRegression(
                max_iter=3000, solver="lbfgs", random_state=42
            )
        }
    ]
}


In [5]:
# Preparar o ambiente do MLFlow e iniciar o experimento

# Configurar o caminho relativo para os artefatos
mlflow.set_tracking_uri("file:./mlruns")

# lista para armazenar os resultados
results = []

# Iniciar o experimento
mlflow.set_experiment("exp_projeto_ciclo_3")

# Run para registrar modelos gerados pelo TPOTClassifier
with mlflow.start_run(run_name="Modelos TPOT Treinados") as main_run:  # Principal
    counter = 0  # Contador para os experimentos
    for params in models["TPOT AutoML"]:  # Itera sobre os parâmetros do TPOT AutoML
        counter += 1
        with mlflow.start_run(run_name=f"{counter}. TPOTClassifier", nested=True):  # Aninhada
            # Instanciar e treinar o TPOTClassifier
            model = TPOTClassifier(**params)
            model.fit(x_train, y_train)

            # Obter o pipeline otimizado
            best_pipeline = model.fitted_pipeline_

            # Avaliar as métricas usando cross_val_score para o pipeline otimizado
            accuracy = cross_val_score(best_pipeline, x_train, y_train, cv=10, scoring='accuracy').mean()
            precision = cross_val_score(best_pipeline, x_train, y_train, cv=10, scoring='precision_weighted').mean()
            recall = cross_val_score(best_pipeline, x_train, y_train, cv=10, scoring='recall_weighted').mean()
            f1 = cross_val_score(best_pipeline, x_train, y_train, cv=10, scoring='f1_weighted').mean()

            # Registrar os parâmetros, métricas e o pipeline otimizado
            for key, value in params.items():
                mlflow.log_param(key, str(value))
            mlflow.log_metric("Accuracy", accuracy)
            mlflow.log_metric("Precision", precision)
            mlflow.log_metric("Recall", recall)
            mlflow.log_metric("F1 Score", f1)
            mlflow.sklearn.log_model(best_pipeline, artifact_path="TPOT_Classifier",
                                                    registered_model_name="TPOT_Classifier",
                                                    input_example=x_test.head(1))

            # Armazenar resultados
            results.append({
                "model": "TPOTClassifier",
                "params": params,
                "Accuracy": accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1 Score": f1,
            })

# Run para registrar modelos gerados pelo StackingClassifier
with mlflow.start_run(run_name="Modelos Stacking Treinados") as main_run:  # Principal
    counter = 0  # Contador para os experimentos
    for params in models["Stacking Classifier"]:  # Itera sobre os parâmetros do Stacking Classifier
        counter += 1
        with mlflow.start_run(run_name=f"{counter}. StackingClassifier", nested=True):  # Aninhada
            # Instanciar e treinar o StackingClassifier
            model = StackingClassifier(**params)
            model.fit(x_train, y_train)

            # Avaliar as métricas usando cross_val_score para o modelo
            accuracy = cross_val_score(model, x_train, y_train, cv=10, scoring='accuracy').mean()
            precision = cross_val_score(model, x_train, y_train, cv=10, scoring='precision_weighted').mean()
            recall = cross_val_score(model, x_train, y_train, cv=10, scoring='recall_weighted').mean()
            f1 = cross_val_score(model, x_train, y_train, cv=10, scoring='f1_weighted').mean()

            # Registrar os parâmetros, métricas e o modelo treinado
            for key, value in params.items():
                mlflow.log_param(key, str(value))
            mlflow.log_metric("Accuracy", accuracy)
            mlflow.log_metric("Precision", precision)
            mlflow.log_metric("Recall", recall)
            mlflow.log_metric("F1 Score", f1)
            mlflow.sklearn.log_model(model, artifact_path="Stacking_Classifier",
                                            registered_model_name="Stacking_Classifier",
                                            input_example=x_test.head(1))

            # Armazenar resultados
            results.append({
                "model": "StackingClassifier",
                "params": params,
                "Accuracy": accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1 Score": f1,
            })

# Selecionar os 3 melhores modelos com base na métrica F1 Score
best_models = sorted(results, key=lambda x: x["F1 Score"], reverse=True)[:3]

print("\nMelhores Modelos:\n")
for model_info in best_models:
    print(model_info)
print("\n\n")


  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Optimization Progress:   0%|          | 0/5200 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7471573220505878

Generation 2 - Current best internal CV score: 0.7471573220505878

Generation 3 - Current best internal CV score: 0.7471573220505878

Generation 4 - Current best internal CV score: 0.7471573220505878

Generation 5 - Current best internal CV score: 0.7493532612570621

Generation 6 - Current best internal CV score: 0.758445398109056

Generation 7 - Current best internal CV score: 0.758445398109056

Generation 8 - Current best internal CV score: 0.7628548477427003

Generation 9 - Current best internal CV score: 0.7634598795432275

Generation 10 - Current best internal CV score: 0.7657878496135113

Generation 11 - Current best internal CV score: 0.7680144133510352

Generation 12 - Current best internal CV score: 0.7680144133510352

Generation 13 - Current best internal CV score: 0.7700339397763596

Generation 14 - Current best internal CV score: 0.7800998350922081

Generation 15 - Current best internal CV score: 0.78009983

Successfully registered model 'TPOT_Classifier'.
Created version '1' of model 'TPOT_Classifier'.


Optimization Progress:   0%|          | 0/9600 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7547222017092071

Generation 2 - Current best internal CV score: 0.7668361570155349

Generation 3 - Current best internal CV score: 0.7668361570155349

Generation 4 - Current best internal CV score: 0.7668361570155349

Generation 5 - Current best internal CV score: 0.7697254713430465

Generation 6 - Current best internal CV score: 0.7697254713430465

Generation 7 - Current best internal CV score: 0.7697254713430465

Generation 8 - Current best internal CV score: 0.7797549717619717

Generation 9 - Current best internal CV score: 0.7797549717619717

Generation 10 - Current best internal CV score: 0.7797549717619717

Generation 11 - Current best internal CV score: 0.7797549717619717

Generation 12 - Current best internal CV score: 0.7797549717619717

Generation 13 - Current best internal CV score: 0.7797549717619717

Generation 14 - Current best internal CV score: 0.7797549717619717

Generation 15 - Current best internal CV score: 0.780161

Registered model 'TPOT_Classifier' already exists. Creating a new version of this model...
Created version '2' of model 'TPOT_Classifier'.


Registered model 'Stacking_Classifier' already exists. Creating a new version of this model...
Created version '5' of model 'Stacking_Classifier'.


Registered model 'Stacking_Classifier' already exists. Creating a new version of this model...
Created version '6' of model 'Stacking_Classifier'.



Melhores Modelos:

{'model': 'TPOTClassifier', 'params': {'generations': 15, 'population_size': 600, 'verbosity': 2, 'scoring': 'f1_weighted', 'random_state': 42, 'n_jobs': -1}, 'Accuracy': 0.7879682601880877, 'Precision': 0.7940591937841426, 'Recall': 0.7879682601880877, 'F1 Score': 0.7868891769628368}
{'model': 'StackingClassifier', 'params': {'estimators': [('rf_pipeline', Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_features=0.8, min_samples_leaf=4,
                                        min_samples_split=5, n_estimators=150,
                                        random_state=42))])), ('et_pipeline', Pipeline(steps=[('classifier',
                 ExtraTreesClassifier(criterion='entropy', max_features=0.65,
                                      min_samples_leaf=2, min_samples_split=3,
                                      n_estimators=150, random_state=42))])), ('gb_pipeline', Pipeline(steps=[('poly_fea

## 💾 Armazenando Melhor Modelo Com Pipeline

In [7]:
# Configurar o tracking URI relativo para MLFlow
mlflow.set_tracking_uri("file:./mlruns")

# Nome do experimento
experiment_name = "exp_projeto_ciclo_3"
mlflow.set_experiment(experiment_name)

# Função para criar e salvar o pipeline completo no MLFlow
def save_pipeline(ml_model, model_name):
    """
    Combina o modelo com um pipeline de pré-processamento e salva no MLFlow.
    """
    # Criar o pipeline com etapas de pré-processamento
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Imputação de valores nulos
        ('scaler', MinMaxScaler()),                   # Normalização dos dados
        ('model', ml_model)                           # Modelo final
    ])
    pipeline.fit(x_train, y_train)  # Ajusta o pipeline aos dados

    # Wrapper para o pipeline, implementando o modelo para ser usado no MLFlow
    class PipelineWrapper(PythonModel):
        def load_context(self, context):
            # Carregar o pipeline treinado
            self.pipeline = pipeline

        def predict(self, context, model_input):
            # Passar os dados pelo pipeline (imputação + escalonamento + modelo)
            return self.pipeline.predict(model_input)

    # Caminho para salvar o modelo
    save_path = "best_model"
    os.makedirs(save_path, exist_ok=True)

    # Salvar o modelo no MLFlow
    mlflow.pyfunc.save_model(path=save_path, python_model=PipelineWrapper())

    print(f"\n{model_name} salvo em: {save_path}")

# Obter todos os runs do experimento, ordenando pelo Accuracy em ordem decrescente
runs = mlflow.search_runs(experiment_names=[experiment_name], order_by=["metrics.`F1 Score` DESC"], max_results=1)

if not runs.empty:
    # Selecionar o melhor run
    best_run = runs.iloc[0]
    best_run_id = best_run["run_id"]

    # Verificar se há histórico de modelo registrado
    if "tags.mlflow.log-model.history" in best_run:
        log_model_history = json.loads(best_run["tags.mlflow.log-model.history"])
        artifact_path = log_model_history[0]["artifact_path"]

        # Carregar o modelo usando o URI relativo
        model_uri = f"runs:/{best_run_id}/{artifact_path}"

        # Verificar o tipo de modelo
        if "Stacking" in artifact_path:
            print("\nModelo Stacking Classifier carregado, combinando com o pré-processamento...")

            # Carregar o modelo Stacking
            loaded_model = mlflow.sklearn.load_model(model_uri)

            # Salvar o pipeline final com Stacking
            save_pipeline(loaded_model, "Pipeline do Stacking Classifier")

        elif "TPOT" in artifact_path:
            print("\nModelo TPOT AutoML carregado, combinando com o pré-processamento...")

            # Carregar o modelo TPOT
            loaded_model = mlflow.sklearn.load_model(model_uri)

            # Salvar o pipeline final com TPOT
            save_pipeline(loaded_model, "Pipeline do TPOT AutoML")

        else:
            print("\nModelo carregado não reconhecido. Salvando com pipeline genérico...")

            # Carregar o modelo genérico
            loaded_model = mlflow.sklearn.load_model(model_uri)

            # Salvar o pipeline genérico
            save_pipeline(loaded_model, "Pipeline Genérico")
    else:
        print("\nNenhum histórico de modelo registrado encontrado para este run.")
else:
    print("\nNenhum run encontrado para o experimento especificado.")


Modelo TPOT AutoML carregado, combinando com o pré-processamento...

Pipeline do TPOT AutoML salvo em: best_model


## 🔋 Carregando o Modelo Salvo

In [11]:
# Carregar o dataset
data = pd.read_csv("water_potability.csv")
data = data.dropna()

# Dividir o dataset em treino e teste
x = data.drop("Potability", axis=1)
y = data["Potability"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=None, stratify=y)

# Caminho para o modelo na pasta raiz
model_path = "./best_model"

# Carregar o modelo diretamente a partir da pasta raiz
loaded_model = mlflow.pyfunc.load_model(model_path)

# Realizar previsões com dados brutos (por exemplo, os dados de teste)
# Como o pipeline foi salvo, a entrada precisa ser passada diretamente para o modelo,
# que já deve cuidar da normalização e outras transformações, se necessário.
test_predictions = loaded_model.predict(x_test)

# Exibir os dados de entrada (X) e as saídas esperadas (y_test) junto com as previsões
print("Dados de Entrada (X - Features de Teste):")
print(x_test.head())  # Exibe as primeiras 5 linhas de X

print("\nSaídas Esperadas (y_test - Potabilidade Esperada):")
print(y_test.head())  # Exibe as primeiras 5 linhas de y_test

print("\nPrevisões do Modelo (y_pred):")
print(test_predictions[:5])  # Exibe as primeiras 5 previsões do modelo

# Calcular e exibir as métricas de desempenho
# Como o modelo já tem pré-processamento interno, não precisamos passar pelo cross-validation manualmente,
# apenas calcular as métricas diretamente com os dados de teste.
accuracy = accuracy_score(y_test, test_predictions)
precision = precision_score(y_test, test_predictions, average='weighted')
recall = recall_score(y_test, test_predictions, average='weighted')
f1 = f1_score(y_test, test_predictions, average='weighted')

# Exibir as métricas
print("\nMétricas do Modelo Carregado:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Dados de Entrada (X - Features de Teste):
             ph    Hardness        Solids  Chloramines     Sulfate  \
2346   7.617033  242.989402  17681.272357     2.855790  298.413238   
309   10.041028  113.831112  16266.434445     7.939074  363.866890   
2823   6.910765  242.091338   7764.438022     8.045870  440.635509   
2895  13.349889  152.776455  18464.900775     6.717973  334.864070   
1639   6.758852  218.153477  22540.775167     7.196435  304.680599   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  
2346    549.987318       10.065225        76.513170   4.299543  
309     557.486061       13.195341        75.233221   3.807563  
2823    487.932310       18.376802        45.306539   4.340149  
2895    450.846369       17.192564        85.883523   2.531075  
1639    382.502700       18.838735        53.815435   3.639972  

Saídas Esperadas (y_test - Potabilidade Esperada):
2346    1
309     1
2823    1
2895    0
1639    1
Name: Potability, dtype: int64

Previsões do 

## 💻 Modelos Registrados no MLflow

In [9]:
import subprocess

# Definir o tracking URI do MLflow
mlflow_tracking_uri = 'file:./mlruns'  # Caminho relativo

mlflow.set_tracking_uri(mlflow_tracking_uri)

# Iniciar o MLflow UI em um subprocesso separado
mlflow_process = subprocess.Popen(["mlflow", "ui", "--host", "127.0.0.1", "--port", "5000"])

# Exibir a URL do MLflow UI
print("MLflow UI está rodando em http://127.0.0.1:5000")

MLflow UI está rodando em http://127.0.0.1:5000


In [13]:
# Parar o subprocesso do MLflow UI
mlflow_process.terminate()

# Confirmar que o MLflow UI foi parado
print("MLflow UI foi parado")

MLflow UI foi parado
